In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import tensorflow as tf
import numpy as np

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import input_data
from libs import models
from libs import label_wav

In [2]:
flags=tf.app.flags

In [3]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','yes,no,up,down,left,right,on,off,stop,go','Wanted Words')
flags.DEFINE_float('validation_percentage',20,'Validation Percentage')
flags.DEFINE_float('testing_percentage',0,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',40,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',20.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC fingerprint')
flags.DEFINE_float('time_shift_ms',200.0,'Range to randomly shift the training audio by in time.')
FLAGS=flags.FLAGS

In [4]:
silence_percentage=10.0
unknown_percentage=10
background_frequency=0.8
background_volume=0.8
save_folder='..\\..\\_inputs\\train'

In [5]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [6]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [7]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings)

..\..\_inputs\raw\train\audio\*\*.wav


W0119 22:54:49.195368  8864 deprecation_wrapper.py:119] From ..\libs\input_data.py:305: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0119 22:54:49.233155  8864 deprecation_wrapper.py:119] From ..\libs\input_data.py:306: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\..\_inputs\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\..\_inputs\raw\train\audio\_background_noise_\dude_miaowing.wav
..\..\_inputs\raw\train\audio\_background_noise_\exercise_bike.wav
..\..\_inputs\raw\train\audio\_background_noise_\pink_noise.wav
..\..\_inputs\raw\train\audio\_background_noise_\running_tap.wav
..\..\_inputs\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 49, 513), dtype=float32)


In [8]:
labels_list=input_data.prepare_words_list(FLAGS.wanted_words.split(','))

In [9]:
with tf.Session() as sess:
    data,one_hot=audio_processor.get_data_raw(-1,0,model_settings,background_frequency,background_volume,FLAGS.time_shift_ms,
                             'training',sess)
one_hot=one_hot.argmax(axis=1)
truth_labels=[]
for i in range(0,one_hot.shape[0]):
    truth_labels.append(labels_list[one_hot[i]])
truth_labels=np.array(truth_labels)

In [13]:
for i in range(truth_labels.shape[0]):
    save_file=os.path.join(save_folder,'test','audio','%s-%d.wav'%(truth_labels[i],i))
    input_data.save_wav_file(save_file,data[i],FLAGS.sample_rate)